In [4]:
from openpyxl import load_workbook
from openpyxl.worksheet.datavalidation import DataValidation
from openpyxl.utils import get_column_letter
import os

In [3]:
# Read all .xlsx files in all subfolders of 'Filtered_By_Region'
all_files = []
for root, dirs, files in os.walk("Filtered_By_Region"):
    for file in files:
        if file.endswith(".xlsx"):
            all_files.append(os.path.join(root, file))

all_files

['Filtered_By_Region\\BARMM\\BARMM_NEWCON.xlsx',
 'Filtered_By_Region\\BARMM\\BARMM_REPAIR.xlsx',
 'Filtered_By_Region\\CAR\\CAR_ELECTRIFICATION.xlsx',
 'Filtered_By_Region\\CAR\\CAR_GABALDON.xlsx',
 'Filtered_By_Region\\CAR\\CAR_HEALTH.xlsx',
 'Filtered_By_Region\\CAR\\CAR_LMS.xlsx',
 'Filtered_By_Region\\CAR\\CAR_NEWCON.xlsx',
 'Filtered_By_Region\\CAR\\CAR_QRF.xlsx',
 'Filtered_By_Region\\CAR\\CAR_REPAIR.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_ALS-CLC.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_ELECTRIFICATION.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_GABALDON.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_HEALTH.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_LMS.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_NEWCON.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_QRF.xlsx',
 'Filtered_By_Region\\CARAGA\\CARAGA_REPAIR.xlsx',
 'Filtered_By_Region\\NCR\\NCR_ELECTRIFICATION.xlsx',
 'Filtered_By_Region\\NCR\\NCR_GABALDON.xlsx',
 'Filtered_By_Region\\NCR\\NCR_HEALTH.xlsx',
 'Filtered_By_Region\\

In [1]:
for file in all_files:
    if "LMS" in file or "NEWCON" in file:
        dropdown = newcon_dropdown_options
    else:
        dropdown = other_dropdown_options
        
    wb = load_workbook(file)
    ws = wb.active

    # Add a new worksheet (or reuse if exists)
    if "DropdownOptions" not in wb.sheetnames:
        dropdown_ws = wb.create_sheet("DropdownOptions")
    else:
        dropdown_ws = wb["DropdownOptions"]
        
    # Write the options into the helper sheet starting at A1
    for i, option in enumerate(dropdown, start=1):
        dropdown_ws.cell(row=i, column=1, value=option)

    # Hide the helper sheet (optional)
    dropdown_ws.sheet_state = 'hidden'

    #  Define the range where options are stored
    options_range = f"DropdownOptions!$A$1:$A${len(dropdown)}"
        
    new_col_index = ws.max_column + 1
    
    ws.insert_cols(new_col_index)
    ws.cell(row=1, column=new_col_index).value = "Status as of July 11, 2025"

    dv = DataValidation(
        type="list",
        formula1=f"={options_range}", # Reference the options in the helper sheet
        allow_blank=True
    )

    # Apply validation
    ws.add_data_validation(dv)
    col_letter = get_column_letter(ws.max_column)
    dv.add(f"{col_letter}2:{col_letter}{ws.max_row}")
    
    #highlight the new column
    for cell in ws[f"{col_letter}"]:
        cell.fill = openpyxl.styles.PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
    #format first row, wrap text, align center, and make the text bold
    for cell in ws[1]:
        cell.font = openpyxl.styles.Font(bold=True)
        cell.alignment = openpyxl.styles.Alignment(wrap_text=True, horizontal='center', vertical='center')
        #freeze row 1
        ws.freeze_panes = ws["A2"]  # Freeze the first row

    for row in ws.iter_rows(min_row=1, max_row=ws.max_row, min_col=1, max_col=ws.max_column):
        for cell in row:
            #add border to all cells
            cell.border = openpyxl.styles.Border(
                left=openpyxl.styles.Side(style='thin'),
                right=openpyxl.styles.Side(style='thin'),
                top=openpyxl.styles.Side(style='thin'),
                bottom=openpyxl.styles.Side(style='thin')
            )
    #autofit column width
    for col in ws.columns:
        max_length = 0
        column = col[0].column_letter  # Get the column letter
        for cell in col:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = (max_length + 2)
        ws.column_dimensions[column].width = adjusted_width
    # SAVE WORKSHEET
    wb.save(file)
    print(f"Updated {file} with new dropdown options in column {col_letter}.")
    
print("All files updated with dropdown options successfully.")

NameError: name 'all_files' is not defined